In [ ]:
from stegoton import *
import matplotlib.pyplot as plt

In [ ]:
claw = setup(ic=2,tfinal=20,num_output_times=100,amp=0.3,solver_type="sharpclaw")
claw.keep_copy = True
claw.solver.cfl_max = 1.5
claw.solver.cfl_desired = 1.45
claw.run()

In [ ]:
def stress(frame):
    """Compute stress from strain and momentum"""
    from clawpack.riemann.nonlinear_elasticity_1D_py import sigma 
    aux = frame.aux
    epsilon = frame.q[0,:]
    stress = sigma(frame.q,aux,{'stress_relation':'exponential'})
    return stress

i=20

q = claw.frames[i].q
xc = claw.frames[0].grid.x.centers
rho = claw.frames[i].aux[0,:]
u = q[1,:]/rho
sigma = stress(claw.frames[i])
plt.plot(xc,sigma)
#plt.xlim(50,100);

In [ ]:
import matplotlib
from matplotlib import animation
matplotlib.rcParams['animation.embed_limit'] = 2**128
from IPython.display import HTML

fig, ax = plt.subplots(figsize=[12,8], nrows=2, ncols=1, sharex=True)
fig.subplots_adjust(hspace=0)

#fig = plt.figure(figsize=[10,6])
#ax = plt.axes(xlim=(xc[0], xc[-1]), ylim=(2.4, 2.9))
ax[0].set_ylim(-0.2,0.25)
ax[1].set_ylim(-0.2,0.4)
ax[0].set_xlim(xc[0],xc[-1])
ax[1].set_xlim(xc[0],xc[-1])
line, = ax[0].plot([], [], lw=2)
line2, = ax[1].plot([], [], lw=2)

def fplot(i):
    t = i*claw.tfinal/claw.num_output_times
    tx = 1.*t + 20
    xl = max(tx-60,0)
    xr = max(tx, 60)

    frame = claw.frames[i]
    eps = frame.q[0,:]
    rho = frame.aux[0,:]
    K = frame.aux[1,:]
    sigma = np.exp(K*eps)-1
    q2 = frame.q[1,:]
    line.set_data(xc, -q2/rho)
    line2.set_data(xc, sigma)
    ax[0].set_title("t= {:.1f}".format(frame.t))
    ax[0].set_ylabel('-u',fontsize=20)
    ax[1].set_ylabel('Stress',fontsize=20)
    #ax[0].set_xlim(xl,xr)
    #ax[1].set_xlim(xl,xr)
    
    fname = 'frame'+str(i).zfill(4)+'.png'
    fig.savefig(fname)
    
    return line,

anim = animation.FuncAnimation(fig, fplot, frames=len(claw.frames), repeat=False)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
claw.solver.dt_old